#  Heart rhythm classification from raw ECG signals

## Notes
Interesting insights from Nature paper published on 21.9.2021
- https://www.nature.com/articles/s41598-021-97118-5?proof=t%3B#Tab6
- https://www.youtube.com/watch?v=3tfin4sSBFQ
- Focus on only two features PR and RT.
- MLP and SVM


## Done
- Verified heartbeat feature extraction transformer by comparing mean beat of sample 0 (np.array_equal and plot) -> looks OK
    - extracted features hb_feat = extractor.fit_transform(...) return shape (num_samples, num_features)
    - mean = hb_feat[0][:180] if no downsampling gives mean beat
- Rpeaks in hb extractor and delineation extractor for sample 0, 1, 2, 3 are the same -> OK

https://ecgwaves.com/topic/ecg-normal-p-wave-qrs-complex-st-segment-t-wave-j-point/

In [1]:
import pandas as pd
import numpy as np
import neurokit2 as nk
import biosppy.signals.ecg as ecg
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib.pyplot as plt
sns.set('talk')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

## Params

In [2]:
train_size = 0.8
sr = 300 #sampling rate

downsampling=False
trim_beginning=False
# sampling rate needs adjustment if downsampling is applied
sampling_divisor = 2 # default: reduce sampling frequency by factor 2
#sr_down = sr/sampling_divisor

# Some matplotlib setting 
plt.rcParams['figure.figsize'] = (30, 20)
plt.rcParams['lines.linewidth'] = 5
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 32
#plt.rcParams['axes.labelsize'] = 48
#plt.rcParams['axes.titlesize'] = 48

## Read data

In [3]:
path = '/home/rapwag01/eth/aml/task2/'

In [4]:
df_train = pd.read_csv(path+'X_train.csv')

In [5]:
df_test = pd.read_csv(path+'X_test.csv')

In [6]:
df_target = pd.read_csv(path+'y_train.csv')

In [7]:
df_train.head()

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205,x206,x207,x208,x209,x210,x211,x212,x213,x214,x215,x216,x217,x218,x219,x220,x221,x222,x223,x224,x225,x226,x227,x228,x229,x230,x231,x232,x233,x234,x235,x236,x237,x238,x239,x240,x241,x242,x243,x244,x245,x246,x247,x248,...,x17592,x17593,x17594,x17595,x17596,x17597,x17598,x17599,x17600,x17601,x17602,x17603,x17604,x17605,x17606,x17607,x17608,x17609,x17610,x17611,x17612,x17613,x17614,x17615,x17616,x17617,x17618,x17619,x17620,x17621,x17622,x17623,x17624,x17625,x17626,x17627,x17628,x17629,x17630,x17631,x17632,x17633,x17634,x17635,x17636,x17637,x17638,x17639,x17640,x17641,x17642,x17643,x17644,x17645,x17646,x17647,x17648,x17649,x17650,x17651,x17652,x17653,x17654,x17655,x17656,x17657,x17658,x17659,x17660,x17661,x17662,x17663,x17664,x17665,x17666,x17667,x17668,x17669,x17670,x17671,x17672,x17673,x17674,x17675,x17676,x17677,x17678,x17679,x17680,x17681,x17682,x17683,x17684,x17685,x17686,x17687,x17688,x17689,x17690,x17691,x17692,x17693,x17694,x17695,x17696,x17697,x17698,x17699,x17700,x17701,x17702,x17703,x17704,x17705,x17706,x17707,x17708,x17709,x17710,x17711,x17712,x17713,x17714,x17715,x17716,x17717,x17718,x17719,x17720,x17721,x17722,x17723,x17724,x17725,x17726,x17727,x17728,x17729,x17730,x17731,x17732,x17733,x17734,x17735,x17736,x17737,x17738,x17739,x17740,x17741,x17742,x17743,x17744,x17745,x17746,x17747,x17748,x17749,x17750,x17751,x17752,x17753,x17754,x17755,x17756,x17757,x17758,x17759,x17760,x17761,x17762,x17763,x17764,x17765,x17766,x17767,x17768,x17769,x17770,x17771,x17772,x17773,x17774,x17775,x17776,x17777,x17778,x17779,x17780,x17781,x17782,x17783,x17784,x17785,x17786,x17787,x17788,x17789,x17790,x17791,x17792,x17793,x17794,x17795,x17796,x17797,x17798,x17799,x17800,x17801,x17802,x17803,x17804,x17805,x17806,x17807,x17808,x17809,x17810,x17811,x17812,x17813,x17814,x17815,x17816,x17817,x17818,x17819,x17820,x17821,x17822,x17823,x17824,x17825,x17826,x17827,x17828,x17829,x17830,x17831,x17832,x17833,x17834,x17835,x17836,x17837,x17838,x17839,x17840,x17841
0,0,-64,-66,-69,-72,-75,-77,-80,-86,-89,-83,-70,-51,-21,34,112,227,360,450,481,452,362,228,108,25,-1,-8,-12,-14,-17,-21,-28,-35,-42,-50,-57,-67,-76,-80,-81,-82,-82,-82,-82,-82,-83,-84,-84,-83,-79,-74,-70,-66,-62,-59,-55,-51,-47,-43,-41,-39,-37,-36,-34,-33,-32,-30,-29,-26,-23,-20,-16,-13,-9,-6,-4,0,2,6,12,23,35,48,59,68,76,85,94,102,109,115,122,129,135,142,150,162,180,192,204,215,225,233,239,242,242,235,225,208,187,168,147,125,98,73,54,39,27,19,11,3,-3,-7,-9,-12,-14,-17,-19,-22,-24,-26,-28,-30,-32,-34,-35,-36,-39,-45,-51,-51,-46,-39,-30,-20,-16,-14,-14,-13,-12,-11,-10,-9,-9,-8,-7,-6,-5,-4,-3,-2,0,3,12,29,52,88,135,189,243,296,334,348,353,349,340,331,322,312,300,266,203,138,88,51,13,-3,-10,-13,-16,-17,-18,-17,-15,-11,-6,-1,5,15,22,28,33,37,40,41,37,29,20,13,6,0,-4,-7,-6,-3,2,9,22,45,68,93,121,149,174,189,192,193,195,196,196,196,195,192,187,181,170,149,129,106,77,51,14,-27,-49,-63,-72,-80,-88,-97,-105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [8]:
# make it arrays as custom transformers only accept np arrays
X = df_train.drop('id', axis=1).values
y = df_target.drop('id', axis=1).values.ravel()
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0, train_size=train_size)

---

## Customised Transformers for Pipeline

In [31]:
class MyCleaning(BaseEstimator, TransformerMixin):
    
    def __init__(self, sampling_rate, detrend_method='locreg', filter_method='neurokit', \
                 trim_beginning=True, downsampling=True, skip_num_samples=540, sampling_divisor=2):
        
        self.sampling_rate = sampling_rate
        self.detrend_method = detrend_method
        self.filter_method = filter_method
        self.order = -1
        self.trim_beginning = trim_beginning
        self.downsampling = downsampling
        self.sampling_divisor = sampling_divisor
        self.skip_num_samples = skip_num_samples
        
        if self.detrend_method == 'constant':
            self.order = 0
        elif self.detrend_method == 'linear':
            self.order = 1
        elif self.detrend_method == 'quadratic':
            self.order = 2
        elif self.detrend_method == 'cubic':
            self.order = 3
        elif self.detrend_method == 'poly10':
            self.order = 10
        elif self.detrend_method == 'trav':
            self.detrend_method='tarvainen2002'      
        elif self.detrend_method == 'loess':
            self.detrend_method = 'loess'


    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        """np.apply_along_axis is slower than for loop.
        Keeping for loop.
        Input X should be numpy array, not pd series. Trying to be consistent with sklearn.
        """
        print('Running cleaning...')
        if self.trim_beginning:
            X = self._trim_beginning(X)
            
        if self.downsampling:
            print(f'You are downsampling data by a factor {self.sampling_divisor}. Be aware to adjust sampling frequency for subsequent transformers!')
            X = self._downsampling(X)

        clean_signals = []
        
        # TODO fix shape if one row only!
        if len(X.shape) < 2:
            raise ValueError('Make sure to reshape array to (1, -1) if you feed in one sample only.')

        for sample in np.arange(X.shape[0]):

            if sample % 50 == 0:
                print(f'cleaning sample {sample}')
            
            # drop nans
            ecg_sample = X[sample]
            ecg_nonans = ecg_sample[~np.isnan(ecg_sample)]
            clean = self._cleaning(ecg_nonans)
            
            # pad array with nans to match previous dimensions
            pad_width = X[sample].shape[0]-clean.shape[0]           
            cleaned_padded = np.pad(clean, pad_width=(0, pad_width), mode='constant', constant_values=np.nan)           
            clean_signals.append(cleaned_padded)
            
        return np.stack(clean_signals)
    
    def _trim_beginning(self, X, skip_num_samples=540):
        """Trim signal at begining and skip n first samples given by skip_num_samples.
        Applied before downsampling if downsampling=True.
        By default skips ~3 heartbeats, i.e. 3x180 samples, where 180 corresponds to the heartbeat extraction
        sample size."""
        X_trimmed = X[:,skip_num_samples:]
        assert X_trimmed.shape[0] == X.shape[0]
        assert X_trimmed.shape[1] == X.shape[1]-skip_num_samples
        
        return X_trimmed
    
    def _downsampling(self, X, sampling_divisor=2):
        """Selects every n-th (sampling_divisor) timestep.
        By default cuts sampling rate into two."""
        X_downsampled = X[:,::sampling_divisor]
        assert X_downsampled.shape[0] == X.shape[0]
        assert X_downsampled.shape[1] == X.shape[1]/sampling_divisor
        return X_downsampled
    
    def _cleaning(self, raw_ecg):
        """
        For detrending: https://neurokit2.readthedocs.io/en/latest/functions.html#neurokit2.signal.signal_detrend
        For filtering: https://neurokit2.readthedocs.io/en/latest/functions.html#neurokit2.ecg.ecg_clean
        """

        detrended = nk.signal_detrend(raw_ecg, order=self.order, method=self.detrend_method, \
                                      window=1.5*100, stepsize=0.02*100)        

        if self.filter_method == 'custom_butterworth':
            cleaned = nk.signal_filter(detrended, sampling_rate=self.sampling_rate, lowcut=2, highcut=9, method='butterworth')
        else:
             # ecg_clean only applies filtering, no detrending
            cleaned = nk.ecg_clean(detrended, sampling_rate=self.sampling_rate, method=self.filter_method)

        return cleaned

In [10]:
#cleaner = MyCleaning(sampling_rate=sr, downsampling=downsampling, trim_beginning=trim_beginning)
#%%time
#new = cleaner.fit_transform(data)

In [30]:
class MyHeartBeatExtractor(BaseEstimator, TransformerMixin):
    """Extracts heartbeats for each cleaned sample separately (num beats x length template).
    Each feature is an aggregation over different heartbeats extracted and has dim of the standard length of one beat (tempalte).
    Template length is the same for all samples but number of beats extracted changes.
    We need to average over number of heartbeats extracted.
    Extracted features correspond to aggregated heartbeat templates for each sample, e.g. if template length is 180, 
    mean beat is an 180-step averaged timeseries, max beat is the maximum amplitude at 180 different timesteps, etc.
    
    """
    def __init__(self, sampling_rate):
        self.sampling_rate = sampling_rate
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('Running hearbeat feature extraction...')
        features = []

        for id_clean in np.arange(X.shape[0]):
            if id_clean % 50 == 0:
                print(f'extracting features from sample {id_clean}')
            sample_features = self._get_features_from_sample(X[id_clean])
            features.append(sample_features)
            
        # should return X_new (num_samples, num_features)
        X_new = np.vstack(features)
        print('final shape of new heartbeat template features:', X_new.shape)
        return X_new

    def _get_rpeaks(self, ecg_cleaned):
        instant_peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=self.sampling_rate)
        return instant_peaks, rpeaks['ECG_R_Peaks']
    
    def _get_features_from_sample(self, ecg_cleaned):
        """Returns a list of aggregated heartbeat features for each sample"""

        ecg_nonans = ecg_cleaned[~np.isnan(ecg_cleaned)]
        _, rpeaks = self._get_rpeaks(ecg_nonans)
        # print('rpeaks', rpeaks)
        beats = ecg.extract_heartbeats(ecg_nonans, rpeaks, self.sampling_rate)['templates']

        # aggregate over heartbeats
        mean_beat = np.mean(beats, axis=0) # average over beats with shape (num beats x standard length per beat) to get mean with dim (length per beat)
        median_beat = np.median(beats, axis=0) # shape (standard heartbeat length,)
        std_beat = np.std(beats, axis=0)
        max_beat = np.max(beats, axis=0)
        min_beat = np.min(beats, axis=0)

        sample_features = [mean_beat, median_beat, std_beat, max_beat, min_beat]
        sample_features = np.hstack(sample_features)
        return sample_features

In [12]:
# adjust sampling frequency
#if downsampling:
#    sr = sr_down
#    print('Resetting sampling frequency')
    
#hb_extractor = MyHeartBeatExtractor(sampling_rate=sr)
#hb_feat = hb_extractor.fit_transform(newt)

In [44]:
a = np.array([np.nan, np.nan, 1,2,3])

In [45]:
np.isnan(a).sum()

2

In [54]:
class MyDelineationExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, sampling_rate, delineation_method='dwt'):
        self.sampling_rate = sampling_rate
        self.delineation_method = delineation_method
           
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('Running delineation feature extraction...')
        features = []
        
        for id_clean in np.arange(X.shape[0]):
            if id_clean % 1 == 0:
                print(f'extracting features from sample {id_clean}')
            
            wave_peaks, rpeaks = self._get_wavepeaks(X[id_clean])
                        
            # get amplitude and timing features and accumulate (mean, median, std, max, min)
            amp_and_timing_features = self._get_sample_features(X[id_clean], wave_peaks, rpeaks)
            # this should be an array of 
            features.append(amp_and_timing_features)
            
        # should return X_new (num_samples, num_features)
        X_new = np.vstack(features)
        print('final shape of new delineation features:', X_new.shape)

        return X_new
    
    def _get_rpeaks(self, ecg_cleaned):
        instant_peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=self.sampling_rate)
        return instant_peaks, rpeaks['ECG_R_Peaks'] 

    def _get_heartrate(self, rpeaks, ecg_cleaned):
        rate = nk.ecg_rate(rpeaks, sampling_rate=self.sampling_rate, desired_length=len(ecg_cleaned))

        return rate
    
    def _get_wavepeaks(self, ecg_cleaned):
              
        ecg_nonans = ecg_cleaned[~np.isnan(ecg_cleaned)]
        _, rpeaks = self._get_rpeaks(ecg_nonans)
        _, waves_peak = nk.ecg_delineate(ecg_nonans, rpeaks, sampling_rate=self.sampling_rate, \
                                         method=self.delineation_method, show=False)
        #print('rpeaks', rpeaks)
        return waves_peak, rpeaks
    
    def _get_sample_features(self, ecg_cleaned, waves_peak, rpeaks):
        """Get a total of 5x12 features for 1)mean, 2)median, 3)std, 4)max, 5)min."""
        waves_peak_nonan = {k:[elem for elem in v if elem is not np.nan] for k,v in waves_peak.items()}
        
        ## amplitude features we want to accumulate over
        # don't want all peaks (no amplitude stats for on-and offsets, just want the PQRST peaks)
        ppeaks = waves_peak_nonan['ECG_P_Peaks']
        qpeaks = waves_peak_nonan['ECG_Q_Peaks']
        speaks = waves_peak_nonan['ECG_S_Peaks']
        tpeaks = waves_peak_nonan['ECG_T_Peaks']
        
        ## timing features we want to accumulate over
        # rr-interval
        rr_interval = np.diff(rpeaks)/sr*1000 # rr interval in ms
        
        # pp-interval
        pp_interval = np.diff(ppeaks)/sr*1000 # pp interval in ms
        
        # qrs duration
        qrs_duration = (np.array(waves_peak['ECG_R_Offsets'])-np.array(waves_peak['ECG_R_Onsets']))/sr*1000 # in ms
        qrs_duration = qrs_duration[~np.isnan(qrs_duration)]
        assert (qrs_duration>0).all()

        # p-wave duration, normal p wave duration 0.12-0.22s
        p_duration = (np.array(waves_peak['ECG_P_Offsets'])-np.array(waves_peak['ECG_P_Onsets']))/sr*1000 # in ms
        p_duration = p_duration[~np.isnan(p_duration)]
        assert (p_duration>0).all()

        # pr segment
        pr_duration = (np.array(waves_peak['ECG_R_Onsets'])-np.array(waves_peak['ECG_P_Onsets']))/sr*1000 # in ms
        pr_duration = pr_duration[~np.isnan(pr_duration)]
        assert (pr_duration>0).all()

        # r-time, time from R onset to R peak, R_onset seems to be the QRS onset: https://neurokit2.readthedocs.io/en/latest/functions.html#neurokit2.ecg_delineate%3E
        rwave_peaktime = rpeaks-waves_peak['ECG_R_Onsets']
        rwave_peaktime = rwave_peaktime[~np.isnan(rwave_peaktime)]
        assert (rwave_peaktime>0).all()
        
        # heartrate = inverse of rr-interval
        heartrate = self._get_heartrate(rpeaks, ecg_cleaned)
        
        mean_feat = []
        median_feat = []
        std_feat = []
        max_feat = []
        min_feat = []

        # calculate stats for all features
        for idx, feat in enumerate([ecg_cleaned[ppeaks], ecg_cleaned[qpeaks], ecg_cleaned[speaks], ecg_cleaned[tpeaks],\
                     ecg_cleaned[rpeaks], rr_interval, pp_interval, qrs_duration, p_duration, \
                     pr_duration, rwave_peaktime, heartrate]):
            
            if np.isnan(feat).sum()>0:
                
                print(f'mean of feat number {idx}: {feat}')
                print(np.mean(feat))
            else:
                print('No Nans')
            mean_feat.append(np.mean(feat)) # mean of rr_interval, mean of pp_interval, ... -> need to hstack
            median_feat.append(np.median(feat))
            std_feat.append(np.std(feat))
            max_feat.append(np.max(feat))
            min_feat.append(np.min(feat))
            
        #print('mean_feat', mean_feat)   
        all_sample_features = np.hstack([[mean_feat], [median_feat], [std_feat], [max_feat], [min_feat]])
        #print('all_sample_features', all_sample_features, all_sample_features.shape, type(all_sample_features))
        
        # column order is all feature means, then medians, stds, max, min
        return all_sample_features


In [55]:
#delin_extractor = MyDelineationExtractor(sampling_rate=sr, delineation_method='dwt')
#delin_feat = delin_extractor.fit_transform(newt)

In [56]:
class MyHRVExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, sampling_rate):
        self.sampling_rate = sampling_rate
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('Running HRV feature extraction...')
        hrvs = []
        for id_clean in np.arange(X.shape[0]):
            if id_clean % 50 == 0:
                print(f'extracting features from sample {id_clean}')
            
            sample_clean = X[id_clean]
            hrv = self._get_hrv(sample_clean)
            hrvs.append(hrv)
        
        hrv_features = np.vstack(hrvs)
        # remove columns where at least one NaN present
        X_new = hrv_features[:,~np.isnan(hrv_features).any(axis=0)]
        
        # should return array of shape (num_samples, num_hrv_features)
        print('final shape of new HRV features:', X_new.shape)
        
        return X_new
    
    def _get_hrv(self, ecg_cleaned):
        
        ecg_nonans = ecg_cleaned[~np.isnan(ecg_cleaned)]
        _, rpeaks = self._get_rpeaks(ecg_nonans)
        
        # TODO add non-linear hrv, not done yet as window size needs adjustment
        hrv_time = nk.hrv_time(rpeaks, sampling_rate=self.sampling_rate)
        hrv_freq = nk.hrv_frequency(rpeaks, sampling_rate=self.sampling_rate, normalize=True)
        #hrv_concat = pd.concat([hrv_time, hrv_freq], axis=1) # add features along axis 1 (horizontically)
        hrv = np.hstack([hrv_time, hrv_freq]) # add features along axis 1 (horizontically)
        return hrv

    def _get_rpeaks(self, ecg_cleaned):
        instant_peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=self.sampling_rate)
        return instant_peaks, rpeaks['ECG_R_Peaks']    

In [57]:
#myhrv = MyHRVExtractor(sampling_rate=sr)
#hrvt = myhrv.transform(newt)

In [58]:
X_train_sub = X_train[:50]
y_train_sub = y_train[:50]

In [59]:
union = FeatureUnion([('heartbeat', MyHeartBeatExtractor(sampling_rate=sr)),
                     ('delineation', MyDelineationExtractor(sampling_rate=sr)),
                     ('hrv', MyHRVExtractor(sampling_rate=sr))])

In [60]:
pipeline = Pipeline([
    ('cleaning', MyCleaning(sampling_rate=sr, downsampling=downsampling, trim_beginning=trim_beginning)),
    ('feature_extraction', union),
    ('scaler', StandardScaler()),
    ('selector', SelectKBest()),
    ('pca', KernelPCA()),
    ('classifier', RandomForestClassifier(class_weight='balanced'))
    ])

In [61]:
pipeline.fit(X_train_sub, y_train_sub)

Running cleaning...
cleaning sample 0
Running hearbeat feature extraction...
extracting features from sample 0
final shape of new heartbeat template features: (50, 900)
Running delineation feature extraction...
extracting features from sample 0
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 1
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 2
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 3
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 4
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 5
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extractin

/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 16
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 17
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 18
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 19
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 20
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 21
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
extracting features from sample 22
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No Nans
No 

Pipeline(steps=[('cleaning',
                 MyCleaning(downsampling=False, sampling_rate=300,
                            trim_beginning=False)),
                ('feature_extraction',
                 FeatureUnion(transformer_list=[('heartbeat',
                                                 MyHeartBeatExtractor(sampling_rate=300)),
                                                ('delineation',
                                                 MyDelineationExtractor(sampling_rate=300)),
                                                ('hrv',
                                                 MyHRVExtractor(sampling_rate=300))])),
                ('scaler', StandardScaler()), ('selector', SelectKBest()),
                ('pca', KernelPCA()),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced'))])

RuntimeWarning: Mean of empty slice. Sample 15 for one feature all nans?

---

In [23]:
X_valid_sub = X_test[:10]
y_pred_sub = pipeline.predict(X_valid_sub)

cleaning sample 0
extracting features from sample 0
final shape of new heartbeat template features: (10, 900)
extracting features from sample 0


/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


final shape of new delineation features: (10, 60)
extracting features from sample 0
final shape of new HRV features: (10, 18)


ValueError: X has 978 features, but StandardScaler is expecting 974 features as input.

In [21]:
# custom transformer to two two step classification (class 3 vs all first)
#class MyOutlierDetection():

---

In [22]:
### VALIDATION - check if features extracted correctly
#check = tvals[0].reshape((1,-1))
#print(check.shape)
#tclean = cleaner.fit_transform(check)
#tclean_nonans = tclean[~np.isnan(tclean)]
#_, trpeaks = nk.ecg_peaks(tclean_nonans, sampling_rate=sr)
#trpeaks = trpeaks['ECG_R_Peaks']
#tbeats = ecg.extract_heartbeats(tclean_nonans, trpeaks, sr)['templates']
## check
#hb_feat = extractor.fit_transform(newt)
#mean0 = hb_feat[0][:180]
#mean_beat = np.mean(tbeats, axis=0)
#np.array_equal(mean0, mean_beat) # should be TRUE